# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

import plotly.express as px

In [2]:
def roll_sma(ssma_list):
    global data
    sema_val = list(pd.DataFrame(ssma_list).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]    
    return(sema_val)

def roll_lma(lsma_list):
    global data
    lema_val = list(pd.DataFrame(lsma_list).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]    
    return(lema_val)

## Read data

In [12]:
source_file_path = f'../data/products/eurusd/eurusd_2021.csv'

# df = pd.read_csv(source_file_path, nrows=10000)
df = pd.read_csv(source_file_path)
print(f'df has {len(df)} records')

reduce_times = 4
df = df.iloc[::reduce_times].reset_index(drop=True)
print(f'df has {len(df)} records')

df.head()

df has 8399947 records
df has 4199974 records
Wall time: 4.96 s


,DateTime,Bid,Ask,Volume
0,20210103 22:00:00.040,1.22396,1.22411,7
1,20210103 22:00:00.434,1.22393,1.22411,1
2,20210103 22:00:00.933,1.22392,1.22409,1
3,20210103 22:00:01.423,1.22389,1.22409,1
4,20210103 22:00:01.854,1.22387,1.22407,1


## Data manipulation

In [4]:
data = {}
data['sma_len'] = 5000
data['lma_len'] = 50000

In [5]:
df['tick'] = (df['Bid'] + df['Ask'])/2
df['spread'] = df['Ask'] - df['Bid']
df = df[['tick', 'spread']]

# Emas ----------------
df['sema'] = df['tick'].rolling(window=data['sma_len']).progress_apply(roll_sma)
df['lema'] = df['tick'].rolling(window=data['lma_len']).progress_apply(roll_lma)

df = df.dropna()

df['ema_diff'] = df['sema'] - df['lema']

df.head()

257500it [04:56, 869.32it/s]
212500it [16:54, 209.55it/s]

Wall time: 21min 50s


,tick,spread,sema,lema,ema_diff
49999,1.215430,0.00002,1.214897,1.216453,-0.001556
50000,1.215485,0.00003,1.214897,1.216453,-0.001555
50001,1.215660,0.00004,1.214897,1.216453,-0.001555
50002,1.215770,0.00004,1.214898,1.216452,-0.001555
50003,1.215600,0.00002,1.214898,1.216452,-0.001555


## Write data to csv

In [6]:
target_file_path = f'data/eurusd/sema_lema_diff1.csv'
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 1.6 s


In [7]:
df['ema_diff'].describe()

count    212500.000000
mean          0.000230
std           0.007251
min          -0.015954
25%          -0.004605
50%           0.000099
75%           0.005363
max           0.013248
Name: ema_diff, dtype: float64

df = pd.read_csv('data/eurusd/ema_diff/sema_lema_diff.csv')
df.head()

df = df[['ema_diff']]
df['ema_diff'] = abs(df['ema_diff'])
df.head()

fig = px.histogram(df, x="ema_diff")
fig.show()

fig = px.line(df, y=["tick", 'sema', 'lema'])
fig.show()